In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(str(x) for x in [0,1])

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.callbacks import *

import numpy as np
import torch
import seaborn as sns
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import auc,roc_curve

import os
print(os.listdir('../mustafa'))

In [ ]:
torch.cuda.set_device(0)

In [ ]:
path='../mustafa'
csv_file='../mustafa/HAM10000_metadata.csv'

In [ ]:
df=pd.read_csv(csv_file).set_index('image_id')
df.head()

In [ ]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [ ]:
df.dx=df.dx.astype('category',copy=True)
df['labels']=df.dx.cat.codes # Convert the labels to numbers
df['lesion']= df.dx.map(lesion_type_dict)
df.head()

In [ ]:
print(df.lesion.value_counts())

In [ ]:
df.loc['ISIC_0027419','lesion']

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize = (10, 5))
sns.countplot(y='lesion',data=df, hue="lesion",ax=ax1)

In [ ]:
class CustomImageItemList(ImageItemList):
    def custom_label(self,df, **kwargs)->'LabelList':
        """Custom Labels from path"""
        file_names=np.vectorize(lambda files: str(files).split('/')[-1][:-4])
        get_labels=lambda x: df.loc[x,'lesion']
        #self.items is an np array of PosixPath objects with each image path
        labels= get_labels(file_names(self.items))
        y = CategoryList(items=labels)
        res = self._label_list(x=self,y=y)
        return res

In [ ]:
def get_data(bs, size):
    train_ds = (CustomImageItemList.from_folder('../mustafa/HAM_part1', extensions='.jpg')
                    .random_split_by_pct(0.15)
                    .custom_label(df)
                    .transform(tfms=get_transforms(flip_vert=True),size=size)
                    .databunch(num_workers=2, bs=bs)
                    .normalize(imagenet_stats))
    return train_ds



In [ ]:
data=get_data(8,224)

In [ ]:
data.classes=list(np.unique(df.lesion))  
data.c= len(np.unique(df.lesion))

In [ ]:
data.show_batch(rows=3)

In [ ]:
learner=create_cnn(data,models.resnet50,metrics=[accuracy], model_dir="../mustafa/tmp/model/")

In [ ]:
learner.loss_func=nn.CrossEntropyLoss()

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(500,1e-3, callbacks=[SaveModelCallback(learner), EarlyStoppingCallback(learner, patience=10)])